# Notebook 7 - Fit a Blender Model for Final Prediction
The purpose of this notebook is to train a regression model on the predictions created from the models in Layer 1 of the stack.

## Import Data and Layer 1 Models and Processors

In [1]:
from sklearn.externals import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import the data to be used for the blender
df_blender_train = joblib.load('../data/df_blender_train.pkl')
df_blender_test = joblib.load('../data/df_blender_test.pkl')
y_blender_train = joblib.load('../data/y_blender_train.pkl')
y_blender_test = joblib.load('../data/y_blender_test.pkl')

# import the layer 1 processors
text_tfidf = joblib.load('../data/4-text_tfidf.pkl')
text_svd = joblib.load('../data/4-text_svd.pkl')
tags_cv = joblib.load('../data/6-tags_cv.pkl')
tags_svd = joblib.load('../data/6-tags_svd.pkl')

# import the layer 1 models for the text
text_rfr = joblib.load('../data/4-text_rfr.pkl')
text_knr = joblib.load('../data/4-text_knr.pkl')
text_bayes = joblib.load('../data/4-text_bayes.pkl')
text_gbr = joblib.load('../data/4-text_gbr.pkl')

# import the layer 1 models for the meta data
meta_rfr = joblib.load('../data/5-meta_rfr.pkl')
meta_knr = joblib.load('../data/5-meta_knr.pkl')
meta_bayes = joblib.load('../data/5-meta_bayes.pkl')
meta_gbr = joblib.load('../data/5-meta_gbr.pkl')

# import the layer 1 models for the hashtags and mentions
tags_rfr = joblib.load('../data/6-tags_rfr.pkl')
tags_knr = joblib.load('../data/6-tags_knr.pkl')
tags_bayes = joblib.load('../data/6-tags_bayes.pkl')
tags_gbr = joblib.load('../data/6-tags_gbr.pkl')


In [3]:
tags_cv = joblib.load('../data/6-tags_cv.pkl')
tags_svd = joblib.load('../data/6-tags_svd.pkl')

## Split the Training and Test Data into Three Sets

In [4]:
# get the text data
X_text_train = df_blender_train.text
X_text_test = df_blender_test.text

# get the meta data
cols = ['year', 'month', 'hour', 'weekday', 'has_hashtag', 'has_mention', 'has_url']
X_meta_train = df_blender_train[cols]
X_meta_test = df_blender_test[cols]

# get the hashtags and mentions data
X_tags_train = df_blender_train.hashtags_and_mentions
X_tags_test = df_blender_test.hashtags_and_mentions

## Train the Blender
Here are the steps to train the blender:
1. Run the data through layer 1 to get predictions
1. Train a model with the predictions as the input
1. Score the model

### 1. Get Layer 1 Predictions

In [5]:
# function to get predictions from layer 1
def get_layer1_preds(data, regr, dataset=None):
    '''
    Runs the data through the layer 1 models to get predictions
    
    Parameters:
        data - feature data to be used in prediction
        regr - fitted regressor
        dataset - string that specifies which kind of layer1 data to process 
                  (text, meta, or tags)
    
    Returns: array with shape = [num_samples] containing predictions 
    '''
    
    if dataset not in ['text', 'meta', 'tags']:
        print("Must specify 'text', 'meta', or 'tags'.")
        return
    
    # transform the text if predicting on text or hashtags and mentions 
    if dataset == 'text':
        sparse = text_tfidf.transform(data)
        data = text_svd.transform(sparse)
    elif dataset == 'tags':
        sparse = tags_cv.transform(data)
        data = tags_svd.transform(sparse)
        
    return regr.predict(data)

#### Loop through all the Layer 1 models and build predictions dataframe

In [6]:
train_preds_dict = {}
test_preds_dict = {}

# loop through the text models
text_models = [('text_rfr_preds', text_rfr), ('text_knr_preds', text_knr), 
               ('text_bayes_preds', text_bayes), ('text_gbr_preds', text_gbr)]
for model in text_models:
    train_preds = get_layer1_preds(X_text_train, model[1], dataset='text')
    test_preds = get_layer1_preds(X_text_test, model[1], dataset='text')
    title = model[0]
    train_preds_dict[title] = train_preds
    test_preds_dict[title] = test_preds
print('All text predictions complete.', end='\n\n')

# loop through the meta models
meta_models = [('meta_rfr_preds', meta_rfr), ('meta_knr_preds', meta_knr), 
               ('meta_bayes_preds', meta_bayes), ('meta_gbr_preds', meta_gbr)]
for model in meta_models:
    train_preds = get_layer1_preds(X_meta_train, model[1], dataset='meta')
    test_preds = get_layer1_preds(X_meta_test, model[1], dataset='meta')
    title = model[0]
    train_preds_dict[title] = train_preds
    test_preds_dict[title] = test_preds
print('All meta predictions complete.', end='\n\n')
    
# loop through the hashtags and mentions models
tags_models = [('tags_rfr_preds', tags_rfr), ('tags_knr_preds', tags_knr), 
               ('tags_bayes_preds', tags_bayes), ('tags_gbr_preds', tags_gbr)]
for model in tags_models:
    train_preds = get_layer1_preds(X_tags_train, model[1], dataset='tags')
    test_preds = get_layer1_preds(X_tags_test, model[1], dataset='tags')
    title = model[0]
    train_preds_dict[title] = train_preds
    test_preds_dict[title] = test_preds
print('All tags predictions complete.', end='\n\n')
    
# write the predictions to a dataframe
X_train_preds = pd.DataFrame(train_preds_dict)
X_test_preds = pd.DataFrame(test_preds_dict)

All text predictions complete.

All meta predictions complete.

All tags predictions complete.



### 2. Train the Blender

In [7]:
from sklearn.linear_model import LinearRegression, Lasso

In [8]:
# function to fit and score a regressor
def fit_and_score(data, regr, return_regr=False):
    '''
    Fits the regressor to the training data and then prints train and test scores.
    
    Parameters:
        data - iterable containing X_train, X_test, y_train, y_test
        regr - instantiated regressor
        return_regr - boolean, option to return the fit regressor (default: False)
    
    Returns: optional, regressor fit to the training data
    '''
    
    print('Regressor: {}'.format(regr))
    
    regr.fit(data[0], data[2])
    print('Train score: {}'.format(regr.score(data[0], data[2])))
    
    print('Test score: {}'.format(regr.score(data[1], data[3])))
    
    if return_regr:
        return regr

In [9]:
data = (X_train_preds, X_test_preds, y_blender_train, y_blender_test)

In [10]:
linreg = LinearRegression()
fit_and_score(data, linreg)

Regressor: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
Train score: 0.4922951265792904
Test score: 0.5071965117148725


In [11]:
joblib.dump(linreg, '../data/7-blender_linreg.pkl')

['../data/7-blender_linreg.pkl']

In [12]:
# get the predicted number of tweets
final_predictions = linreg.predict(X_test_preds)

In [13]:
# create a dataframe of predicted and actual - remember these are log-transformed values
results_dict = {'predicted_scaled': final_predictions, 'actual_scaled': y_blender_test}
results_df = pd.DataFrame(results_dict)
results_df.head()

,actual_scaled,predicted_scaled
87376,1.098612,1.325723
87766,1.098612,1.733436
57239,2.639057,1.722085
34117,0.000000,0.864286
65838,2.772589,2.651178


### Unscale the Retweets
Since the number of retweets were scaled using log(x + 1), they need to be inverted using exp(x) - 1. Then they should be rounded to the nearest integer since there cannot be fractions of a retweet.

In [14]:
results_df['Actual'] = np.expm1(results_df.actual_scaled).apply(lambda x: int(round(x,0)))
results_df['Predicted'] = np.expm1(results_df.predicted_scaled).apply(lambda x: int(round(x,0)))

In [15]:
results_df.head()

,actual_scaled,predicted_scaled,Actual,Predicted
87376,1.098612,1.325723,2,3
87766,1.098612,1.733436,2,5
57239,2.639057,1.722085,13,5
34117,0.000000,0.864286,0,1
65838,2.772589,2.651178,15,13


In [ ]:
joblib.dump(linreg, '../data/7-predictions.pkl')